In [5]:
from core.statements import *
from core.program import *
from core.model import *
from core.queries import *

## Formuły logiczne

Dostępne jest 5 operacji logicznych
- `not()` -- negacja
- `and()` -- koniunkcja
- `or()`  -- alternatywa
- `imp()` -- implikacja
- `eq()`  -- równoważność

Argumenty
- `not` -- jednoargumentowa
- `and` -- dowolna liczba argumentów, dla 0 argumentów zawsze zwraca prawdę
- `or`  -- dowolna liczba argumentów, dla 0 argumentów zawsze zwraca fałsz
- `imp` -- dwuargumentowa
- `eq`  -- dwuargumentowa

Jeśli dana formuła lub zagnieżdżona formuła\
nie jest jedną z powyższych to traktowana jest jako literał\
i jej wartość jest wyszukiwana w zbiorze wartościowań literałów.

Nazwa literału może raczej być dowolna, ale:
- nie może zawierać nawiasów
- znaki białe są usuwane
- wielkość liter nie ma znaczenia (wszystko konwertowane jest do lower case)
- zarezerwowane są dwa specjalne literały `true` i `false`, które zwracają odpowiednio `True` i `False` zawsze

### Przykłady
**Przykład 1**

formuła: `(p v q) ^ ~r`\
inaczej: `(p or q) and not r`\
zapis: `and(or(p, q), not(r))`

**Przykład 2**

formuła: `(p => q) v ~(r <=> q) v (p ^ r)`\
zapis: `or(imp(p, q), not(eq(r, q)), and(p, q))`


In [6]:
from core.expressions import parse_expression

In [7]:
condition = parse_expression('and(or(p, q), not(r))')
values = {
    'p': True,
    'q': False,
    'r': False
}
condition(values)

True

## Przykłady Historyjek i Kwerend

### Przykład 1 - Yale Shooting Problem (Wykład)
Przykład bez użycia agentów -- każda formuła agentowa będzie postaci `true` i dowolna grupa agentów (w tym pusta) będzie mogła
wykonać tą akcję

````{verbatim}
initially alive
initially ~loaded
load causes loaded
shoot causes ~loaded
shoot causes ~alive if loaded
````

In [8]:
fluents = ['alive', 'loaded']   # lista fluentów, jeśli ma być użyta gdzieś indziej to musi być w tej samej kolejności
actions = ['load', 'shoot']     # lista akcji
agents = []                     # lista agentów, jeśli ma być użyta gdzieś indziej to musi być w tej samej kolejności
domain = [
    Initialisation('and(alive, not(loaded))'), # initially alive and not loaded
    Effect('load', 'true', 'loaded'), # load (by true/anyone) causes loaded
    Effect('shoot', 'true', 'not(loaded)'), # shoot (by true/anyone) causes not loaded
    Effect('shoot', 'true', 'not(alive)', 'loaded') # shoot (by true/anyone) causes not alive if loaded
]

model = Model(fluents, actions, agents, domain)

Stany początkowe

In [6]:
model.initial_states

[{'loaded': False, 'alive': True}]

Wszystkie stany

In [9]:
model.states

[{'loaded': False, 'alive': False},
 {'loaded': True, 'alive': False},
 {'loaded': False, 'alive': True},
 {'loaded': True, 'alive': True}]

Przykład wykonania akcji w stanie początkowym

In [10]:
ag = AgentGroup([], agents) # [] to lista agentów w tej grupie (brak agentów)
s0 = model.initial_states[0]
result = model.res('load', ag, s0)
print(s0)
print(result) # res zwraca listę wszystkich stanów wynikowych, tutaj tylko jeden

{'loaded': False, 'alive': True}
[{'loaded': True, 'alive': True}]


In [11]:
s1 = result[0]
result = model.res('shoot', ag, s1)
print(result) # broń nienaładowana, indyk nie żyje

[{'loaded': False, 'alive': False}]


Przykład Kwerendy i Programu

In [12]:
P = Program([('load', ag), ('shoot', ag)]) # P = (load by ag, shoot by ag)
Q = ValueQuery(True, 'and(not(loaded), not(alive))', P) # necessary (true) not loaded and not alive after P (from true => any initial state)
Q.answer(model)

True

### Przykład 2 -- Rzut Monetą, Agenci

Mamy monetę, 
rzut monetą przez agenta A skutkuje 50/50 heads lub tails (not heads)
rzut monetą przez agenta B zawsze skutkuje heads
rzut monetą przez obu agentów jest niemożliwy
rzut monetą przez żadnego z agentów nic nie robi


````{verbatim}
initially heads
toss by A releases heads
toss by B causes heads
toss by A ^ B causes false // impossible toss by A ^ B
````

In [13]:
fluents = ['heads']
actions = ['toss']
agents = ['A', 'B']
domain = [
    Initialisation('not(heads)'),
    Release('toss', 'A', 'heads'), # toss by A releases heads (with no precondtion)
    Effect('toss', 'B', 'heads'),
    Effect('toss', 'and(A, B)', 'false') # toss by A and B causes false (impossible)
]

model = Model(fluents, actions, agents, domain)

In [14]:
s0 = model.initial_states[0]
agentA = AgentGroup(['A'], agents)
agentB = AgentGroup(['B'], agents)
agentsAB = AgentGroup(['A', 'B'], agents)
noAgents = AgentGroup([], agents)

In [15]:
model.res('toss', agentA, s0) # releases, więc obie strony monety możliwe

[{'heads': False}, {'heads': True}]

In [16]:
model.res('toss', agentB, s0) # tylko heads możliwe

[{'heads': True}]

Niemożliwe akcje na ogół wyrzucają błąd

In [17]:
try:
    model.res('toss', agentsAB, s0)
except Exception as e:
    print(e)

Unexecutable action toss by {'a': True, 'b': True} in {'heads': False}


Ale można to obejść (wewnątrz silnika często jest obchodzone bo łatwiej sprawdzić pustą listę niż błąd)

In [18]:
model.res('toss', agentsAB, s0, force_execution=True)

[]

Pustej listy nie należy mylić z brakiem efektów\
Brak efektów skutkuje stanem, z którego zaczęliśmy to znaczy

In [17]:
model.res('toss', noAgents, s0)
# zwracany tylko jeden stan not heads mimo, że w domenie nie ma zdania, które deterministycznie to powoduje
# możliwe jedynie dlatego, że akcja toss bez agentów ma pusty efekt

[{'heads': False}]

Executability Query

In [18]:
P = Program([('toss', agentsAB)])
Q = ExecutabilityQuery(True, P) # necessary executable P (from initial states)
Q.answer(model)

False

In [19]:
P = Program([('toss', agentA)])
Q = ExecutabilityQuery(True, P) # possibly executable P (from initial states)
Q.answer(model)

True

Accessability Query

In [20]:
Q = AccessabilityQuery(True, 'heads', agentA) # necessary accessible heads by agentA (from initial states)
Q.answer(model) # False, bo może zawszy wypadać tails (not heads)

False

In [21]:
Q = AccessabilityQuery(False, 'heads', agentA) # possibly accessible heads by agentA (from initial states)
Q.answer(model) # True bo może wypaść heads

True

In [22]:
Q = AccessabilityQuery(True, 'heads', agentB) # necessary accessible heads by agentB (from initial states)
Q.answer(model) # True bo zawsze wypada heads

True

In [23]:
Q = AccessabilityQuery(False, 'not(heads)', agentB) # possibly accessible not heads by agentB (from initial states)
Q.answer(model) # True bo zbiór pusty to podzbiór agentB, a dla zbioru pustego agentów dostajemy pusty efekt

True

In [24]:
Q = AccessabilityQuery(False, 'not(heads)', agentB, 'heads') # possibly accessible not heads by agentB from heads
Q.answer(model) # False bo z heads, albo zawsze dostaniemy heads jeśli agent B rzuca, lub pusty efekt jeśli nikt nie rzuca

False

### Przykład 3 -- Światło i przełączniki (Wykład)

````{verbatim}
initially switch1 ^ switch2
always light <=> (switch1 <=> switch2)
toggle1 causes switch1 if ~switch1
toggle1 causes ~switch1 if switch1
toggle2 causes switch2 if ~switch2
toggle2 causes ~switch2 if switch2
````

In [25]:
fluents = ['switch1', 'switch2', 'light']
actions = ['toggle1', 'toggle2']
agents = []

domain = [
    Initialisation('and(switch1, switch2)'),
    Constraint('eq(light, eq(switch1, switch2))'), # always...
    Effect('toggle1', 'true', 'switch1', 'not(switch1)'),
    Effect('toggle1', 'true', 'not(switch1)', 'switch1'),
    Effect('toggle2', 'true', 'switch2', 'not(switch2)'),
    Effect('toggle2', 'true', 'not(switch2)', 'switch2'),
]

model = Model(fluents, actions, agents, domain)

In [26]:
s0 = model.initial_states[0]
ag = AgentGroup([], agents)
model.res('toggle1', ag, s0)
# przełączenie switch1 powoduje wyłączenie światła albo wyłączenie drugiego switcha

[{'light': True, 'switch2': False, 'switch1': False},
 {'light': False, 'switch2': True, 'switch1': False}]

Dodajemy `nonintertial light` 

In [27]:
domain = [
    Initialisation('and(switch1, switch2)'),
    Constraint('eq(light, eq(switch1, switch2))'),
    Specification('light'), # noninertial light
    Effect('toggle1', 'true', 'switch1', 'not(switch1)'),
    Effect('toggle1', 'true', 'not(switch1)', 'switch1'),
    Effect('toggle2', 'true', 'switch2', 'not(switch2)'),
    Effect('toggle2', 'true', 'not(switch2)', 'switch2'),
]

model = Model(fluents, actions, agents, domain)

In [28]:
s0 = model.initial_states[0]
ag = AgentGroup([], agents)
model.res('toggle1', ag, s0)
# teraz przełączenie toggle1 jedynie wyłącza światło

[{'light': False, 'switch2': True, 'switch1': False}]

### Przykład 4 -- Pracownicy (Przykład 3 z dokumentu)

`atLeastTwoWorkers` := $(P_1 \wedge P_2)\vee(P_2 \wedge P_3)\vee (P_1\wedge P_3) $\
`atLeastThreeWorkers` := $P_1\wedge P_2\wedge P_3$ 

````{verbatim}
initially ~block1, ~block2
always block2=>block1
addBlock by atLeastTwoWorkers causes block1 if ~block1
addBlock by atLeastThreeWorkers causes block2 if block1 ^ ~block2
````

In [3]:
fluents = ['block1', 'block2']
actions = ['addBlock']
agents = ['p1', 'p2', 'p3']

domain = [
    Initialisation('and(not(block1), not(block2))'),
    Constraint('imp(block2, block1)'),
    Effect('addBlock', 'or(and(p1, p2), and(p2, p2), and(p1, p3))', 'block1', 'not(block1)'),
    Effect('addBlock', 'and(p1, p2, p3)', 'block2', 'and(block1, not(block2))'), 
]

model = Model(fluents, actions, agents, domain)

agentsP1P2 = AgentGroup(['p1', 'p2'], agents)
agentsP1P2P3 = AgentGroup(['p1', 'p2', 'p3'], agents)

NameError: name 'Initialisation' is not defined

In [3]:
Q = AccessabilityQuery(False, 'block2', agentsP1P2)
Q.answer(model) # dwóch praconików nigdy nie postawi dwóch bloków

False

In [4]:
Q = AccessabilityQuery(False, 'block2', agentsP1P2P3)
Q.answer(model) # trzech pracowników jest w stanie postawić dwa bloki

True

In [6]:
P = Program([('addBlock', agentsP1P2), ('addBlock', agentsP1P2P3)])
Q = SufficiencyQuery(agentsP1P2, P)
print(Q.answer(model))
Q = SufficiencyQuery(agentsP1P2P3, P)
print(Q.answer(model))

False
True


### Przykład 5 -- Uciekający Indyk

zmodyfikowany Yale Shooting Problem. Fred jest też agentem, który może zacząć uciekać. Jeśli ucieka Bill nie zawsze będzie w stanie go trafić.

````{verbatim}
initially alive
initially ~loaded
initially ~fredRunning
always fredRunning => alive
load by Bill causes loaded
shoot by Bill causes ~loaded 
shoot by Bill causes ~alive if loaded ^ ~fredRunning
shoot by Bill releases alive if alive ^ loaded ^ fredRunning
run by Fred causes fredRunning if alive
````

In [83]:
fluents = ['alive', 'loaded', 'fredRunning']
actions = ['load', 'shoot', 'run']
agents = ['Bill', 'Fred']

domain = [
    Initialisation('and(alive, not(loaded), not(fredRunning))'),
    Constraint('imp(fredRunning, alive)'),
    Effect('load', 'Bill', 'loaded'),
    Effect('shoot', 'Bill', 'not(loaded)'),
    Effect('shoot', 'Bill', 'not(alive)', 'and(loaded, not(fredRunning))'),
    Release('shoot', 'Bill', 'alive', 'and(loaded, alive, fredRunning)'),
    Release('shoot', 'Bill', 'fredRunning', 'and(loaded, alive, fredRunning)'),
    Effect('run', 'Fred', 'fredRunning', 'alive')
]

model = Model(fluents, actions, agents, domain)

In [84]:
G = AgentGroup(agents, agents)
Q = AccessabilityQuery(False, 'not(alive)', G, 'and(alive, not(loaded), fredRunning)')
Q.answer(model)

True

In [77]:
fluents = ['alive', 'loaded', 'fredRunning']
actions = ['load', 'shoot', 'run']
agents = ['Bill', 'Fred']

domain = [
    Initialisation('and(alive, not(loaded), not(fredRunning))'),
    Constraint('imp(fredRunning, alive)'),
    Effect('load', 'Bill', 'loaded'),
    Release('load', 'Bill', 'fredRunning', 'not(fredRunning)'),
    Effect('shoot', 'Bill', 'not(loaded)'),
    Effect('shoot', 'Bill', 'not(alive)', 'and(loaded, not(fredRunning))'),
    Release('shoot', 'Bill', 'alive', 'and(loaded, alive, fredRunning)'),
    Release('shoot', 'Bill', 'fredRunning', 'and(loaded, alive, fredRunning)'),
]

model = Model(fluents, actions, agents, domain)

In [82]:
G = AgentGroup(agents, agents)
Q = AccessabilityQuery(True, 'not(alive)', G)
Q.answer(model)

False